In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# 1. 데이터 로드

In [2]:
# 데이터셋 다운로드: httphttps://archive.ics.uci.edu/ml/datasets/online+retail
df = pd.read_excel('./data/online+retail/Online Retail.xlsx')

# 2. 전처리: 결측 CustomID 제거

In [4]:
df = df[df['CustomerID'].notnull()]

# 3. 구매 금액 계산: (Quantity * UnitPrice)

In [6]:
df['purchase_amount'] = df['Quantity'] * df['UnitPrice']

C:\Users\PC\AppData\Local\Temp\ipykernel_33692\3456016273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purchase_amount'] = df['Quantity'] * df['UnitPrice']


# 4. 날짜 변수 변환: InvoiceDate를 datetime 형식으로 변경

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

C:\Users\PC\AppData\Local\Temp\ipykernel_33692\3633860036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


# 5. 캠페인 전후 구분

In [10]:
# 예: 캠페인 시작일을 2011-12-01로 가정
campaign_start = pd.to_datetime('2011-12-01')
df['time'] = (df['InvoiceDate'] >= campaign_start).astype(int)

C:\Users\PC\AppData\Local\Temp\ipykernel_33692\3351291811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = (df['InvoiceDate'] >= campaign_start).astype(int)


# 6. 처리군(treated) vs 통제군(control) 구분

In [12]:
# 예: 고객별로 무작위로 쿠폰 발행 여부를 할당 (처리 확률 50%)
unique_cutomers = df['CustomerID'].unique()
np.random.seed(42)  # 재현성을 위해 시드 고정
treatment_assignment = {customer: np.random.binomial(1, 0.5) for customer in unique_cutomers}
df['treated_flag'] = df['CustomerID'].map(treatment_assignment)

C:\Users\PC\AppData\Local\Temp\ipykernel_33692\115059649.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['treated_flag'] = df['CustomerID'].map(treatment_assignment)


# 7. DiD 분석을 위해 고객별, 시점별로 구매액 집계

In [14]:
agg_df = df.groupby(['CustomerID', 'treated_flag', 'time'])['purchase_amount'].sum().reset_index()

# 8. Difference-in-Differences 회귀 분석

In [16]:
# 모형: purchase_amount ~ treated_flag + time + treated_flag:time
model = smf.ols(formula='purchase_amount ~ treated_flag + time + treated_flag:time', data=agg_df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        purchase_amount   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     6.608
Date:                Sat, 15 Feb 2025   Prob (F-statistic):           0.000188
Time:                        19:08:02   Log-Likelihood:                -51810.
No. Observations:                5017   AIC:                         1.036e+05
Df Residuals:                    5013   BIC:                         1.037e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          1763.5531    159.16

# 결과 해석
- **Intercept (1763.55):**
    
    캠페인 이전, 통제군의 평균 구매액이 약 1,763.55임을 나타냅니다.
    
- **treated_flag (147.08, p=0.513):**
    
    캠페인 이전에 처리군이 통제군보다 평균적으로 약 147.08만큼 높은 구매액을 보였으나, 통계적으로 유의하지 않습니다.
    
- **time (-1270.37, p=0.003):**
    
    캠페인 이후, 통제군의 구매액이 약 1,270.37만큼 감소했음을 의미하며, 이는 통계적으로 유의한 결과입니다.
    
    - 이 결과는 계절성 또는 외부 요인에 의한 전반적 구매 감소 가능성을 시사할 수 있습니다.
- **treated_flag:time (-135.00, p=0.824):**
    
    DiD 효과로, 캠페인 이후 처리군과 통제군 간의 추가적인 차이를 의미합니다.
    
    - 계수가 -135로 나타났으나, p-value가 높아 유의하지 않으므로, 캠페인이 처리군의 구매액에 별다른 인과적 영향을 미쳤다고 보기는 어렵습니다.

- **추가 고려사항:**
    - 전체 모형의 설명력이 낮은 편(R² ≈ 0.004)이며, 이는 모델에 포함된 변수 외에도 구매액에 영향을 주는 다른 요인들이 있을 가능성을 암시합니다.
    - 데이터의 분포(예: 왜도와 첨도)가 매우 치우쳐 있을 수 있으므로, 추가 전처리나 로그 변환 등의 조치가 필요할 수 있습니다.